# Adding Helical Features

In [1]:
import MDAnalysis

In [2]:
import MDAnalysis.analysis

In [3]:
from __future__ import print_function, division, absolute_import
from six.moves import range, zip

import os

import numpy as np

import MDAnalysis
from MDAnalysis.lib.log import ProgressBar
from MDAnalysis.lib import mdamath

import warnings
import logging
logger = logging.getLogger("MDAnalysis.analysis.helanal")


def center(coordinates):
    """Return the geometric center (centroid) of the coordinates.

    Coordinates must be "list of cartesians", i.e. a Nx3 array.
    """
    return np.mean(coordinates, axis=0)

def vecnorm(a):
    """Return a/|a|"""
    return a / mdamath.norm(a)

def wrapangle(angle):
    """Wrap angle (in radians) to be within -pi < angle =< pi"""
    if angle > np.pi:
        angle -= 2 * np.pi
    elif angle <= -np.pi:
        angle += 2 * np.pi
    return angle

def sample_sd(a, dummy):
    return np.std(a, ddof=1)

def mean_abs_dev(a, mean_a=None):
    if mean_a is None:
        mean_a = np.mean(a)
    return np.mean(np.fabs(a - mean_a))



def helanal_trajectory(universe, select="name CA",
                       begin=None, finish=None,
                       matrix_filename="bending_matrix.dat",
                       origin_pdbfile="origin.pdb",
                       summary_filename="summary.txt",
                       screw_filename="screw.xvg",
                       tilt_filename="local_tilt.xvg",
                       fitted_tilt_filename="fit_tilt.xvg",
                       bend_filename="local_bend.xvg",
                       twist_filename="unit_twist.xvg",
                       prefix="helanal_", ref_axis=None,
                       verbose=False):
    """Perform HELANAL helix analysis on all frames in `universe`.

    Parameters
    ----------
    universe : Universe
    select : str (optional)
        selection string that selects Calpha atoms [``"name CA"``]
    begin : float (optional)
        start analysing for time (ps) >= *begin*; ``None`` starts from the
        beginning [``None``]
    finish : float (optional)
        stop analysis for time (ps) =< *finish*; ``None`` goes to the
        end of the trajectory [``None``]
    matrix_filename : str (optional)
        Output file- bending matrix [``"bending_matrix.dat"``]
    origin_pdbfile : str (optional)
        Output file- origin pdb file [``"origin.pdb"``]
    summary_filename : str (optional)
        Output file- all of the basic data [``"summary.txt"``]
    screw_filename : str (optional)
        Output file- local tilts of individual residues from 2 to n-1
        [``"screw.xvg"``]
    tilt_filename : str (optional)
        Output file- tilt of line of best fit applied to origin axes
        [``"local_tilt.xvg"``]
    bend_filename : str (optional)
        Output file- local bend angles between successive local helix axes
        [``"local_bend.xvg"``]
    twist_filename : str (optional)
        Output file- local unit twist between successive helix turns
        [``"unit_twist.xvg"``]
    prefix : str (optional)
        Prefix to add to all output file names; set to ``None`` to disable
        [``"helanal__"``]
    ref_axis : array_like (optional)
        Calculate tilt angle relative to the axis; if ``None`` then ``[0,0,1]``
        is chosen [``None``]
    verbose : bool (optional)
        Toggle diagnostic outputs. [``True``]

    Raises
    ------
    ValueError
          If the specified start (begin) time occurs after the end of the
          trajectory object.
          If the specified finish time precedes the specified start time or
          current time stamp of trajectory object.

    Notes
    -----
    Only a single helix is analyzed. Use the selection to specify the helix,
    e.g. with "name CA and resid 1:20" or use start=1, stop=20.
    """
    if ref_axis is None:
        ref_axis = np.array([0., 0., 1.])
    else:
        # enable MDA API so that one can use a tuple of atoms or AtomGroup with
        # two atoms
        ref_axis = np.asarray(ref_axis)

    ca = universe.select_atoms(select)
    start, end = ca.resids[[0, -1]]
    trajectory = universe.trajectory

    # Validate user supplied begin / end times
    traj_end_time = trajectory.ts.time + trajectory.totaltime

    if begin is not None:
        if traj_end_time < begin:
            # Begin occurs after the end of the trajectory, throw error
            msg = ("The input begin time ({0} ps) occurs after the end "
                   "of the trajectory ({1} ps)".format(begin, traj_end_time))
            raise ValueError(msg)
        elif trajectory.ts.time > begin:
            # Begin occurs before trajectory start, warn and reset
            msg = ("The input begin time ({0} ps) precedes the starting "
                   "trajectory time --- Setting starting frame to 0".format(
                    begin))
            warnings.warn(msg)
            logger.warning(msg)
            start_frame = None
        else:
            start_frame = int(np.ceil((begin - trajectory.ts.time)
                                      / trajectory.ts.dt))
    else:
        start_frame = None

    if finish is not None:
        if (begin is not None) and (begin > finish):
            # finish occurs before begin time
            msg = ("The input finish time ({0} ps) precedes the input begin "
                   "time ({1} ps)".format(finish, begin))
            raise ValueError(msg)
        elif trajectory.ts.time > finish:
            # you'd be starting with a finish time(in ps) that has already
            # passed or is not available
            msg = ("The input finish time ({0} ps) precedes the current "
                   "trajectory time ({1} ps)".format(finish, trajectory.time))
            raise ValueError(msg)
        elif traj_end_time < finish:
            # finish time occurs after the end of trajectory, warn
            msg = ("The input finish time ({0} ps) occurs after the end of "
                   "the trajectory ({1} ps). Finish time will be set to the "
                   "end of the trajectory".format(finish, traj_end_time))
            warnings.warn(msg)
            logger.warning(msg)
            end_frame = None
        else:
            # To replicate the original behaviour of break when
            # trajectory.time > finish, we add 1 here.
            end_frame = int(np.floor((finish - trajectory.ts.time)
                            // trajectory.ts.dt) + 1)
    else:
        end_frame = None

    start_frame, end_frame, frame_step = trajectory.check_slice_indices(
                                          start_frame, end_frame, 1)
    n_frames = len(range(start_frame, end_frame, frame_step))

    if start is not None and end is not None:
        logger.info("Analysing from residue %d to %d", start, end)
    elif start is not None and end is None:
        logger.info("Analysing from residue %d to the C termini", start)
    elif start is None and end is not None:
        logger.info("Analysing from the N termini to %d", end)
    logger.info("Analysing %d/%d residues", ca.n_atoms, universe.atoms.n_residues)

    if prefix is not None:
        prefix = str(prefix)
        matrix_filename = prefix + matrix_filename
        origin_pdbfile = prefix + origin_pdbfile
        summary_filename = prefix + summary_filename
        screw_filename = prefix + screw_filename
        tilt_filename = prefix + tilt_filename
        fitted_tilt_filename = prefix + fitted_tilt_filename
        bend_filename = prefix + bend_filename
        twist_filename = prefix + twist_filename
    
    backup_file(matrix_filename)
    backup_file(origin_pdbfile)
    backup_file(summary_filename)
    backup_file(screw_filename)
    backup_file(tilt_filename)
    backup_file(fitted_tilt_filename)
    backup_file(bend_filename)
    backup_file(twist_filename)

    global_height = []
    global_twist = []
    global_rnou = []
    global_bending = []
    global_bending_matrix = []
    global_tilt = []
    global_fitted_tilts = []
    global_screw = []

    for ts in ProgressBar(trajectory[start_frame:end_frame:frame_step],
                          verbose=verbose, desc="Helix analysis"):

        frame = ts.frame
        ca_positions = ca.positions
        twist, bending_angles, height, rnou, origins, local_helix_axes, local_screw_angles = \
            main_loop(ca_positions, ref_axis=ref_axis)

        origin_pdb(origins, origin_pdbfile)

        #calculate local bending matrix( it is looking at all i, j combinations)
        if len(global_bending_matrix) == 0:
            global_bending_matrix = [[[] for item in local_helix_axes] for item in local_helix_axes]

        for i in range(len(local_helix_axes)):
            for j in range(i + 1, len(local_helix_axes)):
                angle = np.rad2deg(np.arccos(np.dot(local_helix_axes[i], local_helix_axes[j])))
                global_bending_matrix[i][j].append(angle)
                #global_bending_matrix[j][i].append(angle)
                #global_bending_matrix[i][i].append(0.)

        fit_vector, fit_tilt = vector_of_best_fit(origins)
        global_height += height
        global_twist += twist
        global_rnou += rnou
        #global_screw.append(local_screw_angles)
        global_fitted_tilts.append(np.rad2deg(fit_tilt))

        #print out rotations across the helix to a file
        with open(twist_filename, "a") as twist_output:
            print(frame, end='', file=twist_output)
            for loc_twist in twist:
                print(loc_twist, end='', file=twist_output)
            print("", file=twist_output)

        with open(bend_filename, "a") as bend_output:
            print(frame, end='', file=bend_output)
            for loc_bend in bending_angles:
                print(loc_bend, end='', file=bend_output)
            print("", file=bend_output)

        with open(screw_filename, "a") as rot_output:
            print(frame, end='', file=rot_output)
            for rotation in local_screw_angles:
                print(rotation, end='', file=rot_output)
            print("", file=rot_output)

        with open(tilt_filename, "a") as tilt_output:
            print(frame, end='', file=tilt_output)
            for tilt in local_helix_axes:
                print(np.rad2deg(mdamath.angle(tilt, ref_axis)),
                      end='', file=tilt_output)
            print("", file=tilt_output)

        with open(fitted_tilt_filename, "a") as tilt_output:
            print(frame, np.rad2deg(fit_tilt), file=tilt_output)

        if len(global_bending) == 0:
            global_bending = [[] for item in bending_angles]
            #global_tilt = [ [] for item in local_helix_axes ]
        for store, tmp in zip(global_bending, bending_angles):
            store.append(tmp)
        #for store,tmp in zip(global_tilt,local_helix_axes): store.append(mdamath.angle(tmp,ref_axis))

    twist_mean, twist_sd, twist_abdev = stats(global_twist)
    height_mean, height_sd, height_abdev = stats(global_height)
    rnou_mean, rnou_sd, rnou_abdev = stats(global_rnou)
    ftilt_mean, ftilt_sd, ftilt_abdev = stats(global_fitted_tilts)

    bending_statistics = [stats(item) for item in global_bending]
    #tilt_statistics =    [ stats(item) for item in global_tilt]

    bending_statistics_matrix = [[stats(col) for col in row] for row in global_bending_matrix]
    with open(matrix_filename, 'w') as mat_output:
        print("Mean", file=mat_output)
        for row in bending_statistics_matrix:
            for col in row:
                formatted_angle = "{0:6.1f}".format(col[0])
                print(formatted_angle, end='', file=mat_output)
            print('', file=mat_output)

        print('\nSD', file=mat_output)
        for row in bending_statistics_matrix:
            for col in row:
                formatted_angle = "{0:6.1f}".format(col[1])
                print(formatted_angle, end='', file=mat_output)
            print('', file=mat_output)

        print("\nABDEV", file=mat_output)
        for row in bending_statistics_matrix:
            for col in row:
                formatted_angle = "{0:6.1f}".format(col[2])
                print(formatted_angle, end='', file=mat_output)
            print('', file=mat_output)

    logger.info("Height: %g  SD: %g  ABDEV: %g  (Angstroem)", height_mean, height_sd, height_abdev)
    logger.info("Twist: %g  SD: %g  ABDEV: %g", twist_mean, twist_sd, twist_abdev)
    logger.info("Residues/turn: %g  SD: %g  ABDEV: %g", rnou_mean, rnou_sd, rnou_abdev)
    logger.info("Fitted tilt: %g  SD: %g  ABDEV: %g", ftilt_mean, ftilt_sd, ftilt_abdev)
    logger.info("Local bending angles:")
    residue_statistics = list(zip(*bending_statistics))
    measure_names = ["Mean ", "SD   ", "ABDEV"]
    if start is None:
        output = " ".join(["{0:8d}".format(item)
                           for item in range(4, len(residue_statistics[0]) + 4)])
    else:
        output = " ".join(["{0:8d}".format(item)
                           for item in range(start + 3, len(residue_statistics[0]) + start + 3)])
    logger.info("ResID %s", output)
    for measure, name in zip(residue_statistics, measure_names):
        output = str(name) + " "
        output += " ".join(["{0:8.1f}".format(residue) for residue in measure])
        logger.info(output)

    with open(summary_filename, 'w') as summary_output:
        print("Height:", height_mean, "SD", height_sd, "ABDEV", height_abdev, '(nm)', file=summary_output)
        print("Twist:", twist_mean, "SD", twist_sd, "ABDEV", twist_abdev,
              file=summary_output)
        print("Residues/turn:", rnou_mean, "SD", rnou_sd, "ABDEV", rnou_abdev,
              file=summary_output)
        print("Local bending angles:", file=summary_output)
        residue_statistics = list(zip(*bending_statistics))
        measure_names = ["Mean ", "SD   ", "ABDEV"]
        print("ResID", end='', file=summary_output)
        if start is None:
            for item in range(4, len(residue_statistics[0]) + 4):
                output = "{0:8d}".format(item)
                print(output, end='', file=summary_output)
        else:
            for item in range(start + 3, len(residue_statistics[0]) + start + 3):
                output = "{0:8d}".format(item)
                print(output, end='', file=summary_output)
        print('', file=summary_output)

        for measure, name in zip(residue_statistics, measure_names):
            print(name, end='', file=summary_output)
            for residue in measure:
                output = "{0:8.1f}".format(residue)
                print(output, end='', file=summary_output)
            print('', file=summary_output)


def tilt_correct(number):
    """Changes an angle (in degrees) so that it is between 0º and 90º"""
    if number < 90.:
        return number
    else:
        return 180. - number


def backup_file(filename):
    if os.path.exists(filename):
        target_name = "#" + filename
        failure = True
        if not os.path.exists(target_name):
            os.rename(filename, target_name)
            failure = False
        else:
            for i in range(20):
                alt_target_name = target_name + "." + str(i)
                if os.path.exists(alt_target_name):
                    continue
                else:
                    os.rename(filename, alt_target_name)
                    failure = False
                    break
        if failure:
            raise IOError("Too many backups. Clean up and try again")


def stats(some_list):
    if len(some_list) == 0:
        return [0, 0, 0]
    list_mean = np.mean(some_list)
    list_sd = sample_sd(some_list, list_mean)
    list_abdev = mean_abs_dev(some_list, list_mean)
    return [list_mean, list_sd, list_abdev]


def helanal_main(pdbfile, select="name CA", ref_axis=None):
    universe = MDAnalysis.Universe(pdbfile)
    ca = universe.select_atoms(select)

    print("Analysing {} CA atoms / {} residues".format(ca.n_atoms, universe.atoms.n_residues))
    print("CA positions:", ca.positions.shape)
    twist, bending_angles, height, rnou, origins, local_helix_axes, local_screw_angles = \
        main_loop(ca.positions, ref_axis=ref_axis)

    max_angle = np.max(bending_angles)
    mean_angle = np.mean(bending_angles)
    
    #sd calculated using n-1 to replicate original fortran- assumes a limited sample so uses the sample standard
    
    # deviation
    sd_angle = sample_sd(bending_angles, mean_angle)
    mean_absolute_deviation_angle = mean_abs_dev(bending_angles, mean_angle)
    
    #Average helical parameters
    mean_twist = np.mean(twist)
    sd_twist = sample_sd(twist, mean_twist)
    abdev_twist = mean_abs_dev(twist, mean_twist)
    #TESTED-average twists
    #print mean_twist, sd_twist, abdev_twist
    mean_rnou = np.mean(rnou)
    sd_rnou = sample_sd(rnou, mean_rnou)
    abdev_rnou = mean_abs_dev(rnou, mean_rnou)
    #TESTED-average residues per turn
    #print mean_rnou, sd_rnou, abdev_rnou
    mean_height = np.mean(height)
    sd_height = sample_sd(height, mean_height)
    abdev_height = mean_abs_dev(height, mean_height)
    #TESTED- average rises

    #calculate best fit vector and tilt of said vector
    fit_vector, fit_tilt = vector_of_best_fit(origins)

    data = {
        'Height': np.array([mean_height, sd_height, abdev_height]),
        'Twist': np.array([mean_twist, sd_twist, abdev_twist]),
        'Residues/turn': np.array([mean_rnou, sd_rnou, abdev_rnou]),
        'Local bending angles': np.asarray(bending_angles),
        'Unit twist angles': np.asarray(twist),
        'Best fit tilt': fit_tilt,
        'Rotation Angles': np.asarray(local_screw_angles),
        }
    return data


def origin_pdb(origins, pdbfile):
    """Write origins to PDB (multi-frame).

    This PDB can be loaded together with the trajectory into, e.g. VMD_, to
    view the helix axis together with all the atoms.
    """
    with open(pdbfile, 'a') as output:
        i = 1
        for xyz in origins:
            tmp = "ATOM    {0:3d}  CA  ALA   {1:3d}    {2:8.3f}{3:8.3f}{4:8.3f}  1.00  0.00".format(i, i, xyz[0], xyz[1], xyz[2])
            print(tmp, file=output)
            i += 1
        print("TER\nENDMDL", file=output)

        
def main_loop(positions, ref_axis=None):
    if ref_axis is None:
        ref_axis = np.array([0., 0., 1.])
    else:
        ref_axis = np.asarray(ref_axis)
    twist = []
    rnou = []
    height = []
    origins = [[0., 0., 0.] for item in positions[:-2]]
    local_helix_axes = []
    location_rotation_vectors = []
    for i in range(len(positions) - 3):
        vec12 = positions[i + 1] - positions[i]
        vec23 = positions[i + 2] - positions[i + 1]
        vec34 = positions[i + 3] - positions[i + 2]

        dv13 = vec12 - vec23
        dv24 = vec23 - vec34

        # direction of the local helix axis
        current_uloc = vecnorm(np.cross(dv13, dv24))
        local_helix_axes.append(current_uloc)

        dmag = mdamath.norm(dv13)
        emag = mdamath.norm(dv24)

        costheta = np.dot(dv13, dv24) / (dmag * emag)
        #rnou is the number of residues per turn
        current_twist = np.arccos(costheta)
        twist.append(np.rad2deg(current_twist))
        rnou.append(2 * np.pi / current_twist)
        #radius of local helix cylinder radmag

        costheta1 = 1.0 - costheta
        radmag = (dmag * emag) ** 0.5 / (2 * costheta1)

        #Height of local helix cylinder
        current_height = np.dot(vec23, current_uloc)
        height.append(current_height)

        dv13 = vecnorm(dv13)
        dv24 = vecnorm(dv24)

        #record local rotation
        location_rotation_vectors.append(dv13)

        rad = [radmag * item for item in dv13]
        current_origin = [(item[0] - item[1]) for item in zip(positions[i + 1], rad)]
        origins[i] = current_origin

        rad = [radmag * item for item in dv24]
        current_origin = [(item[0] - item[1]) for item in zip(positions[i + 2], rad)]
        origins[i + 1] = current_origin
    
    #Record final rotation vector
    location_rotation_vectors.append(dv24)

    #local bending angles (eg i > i+3, i+3 > i+6)

    bending_angles = [0 for item in range(len(local_helix_axes) - 3)]
    for axis in range(len(local_helix_axes) - 3):
        angle = np.arccos(np.dot(local_helix_axes[axis], local_helix_axes[axis + 3]))
        bending_angles[axis] = np.rad2deg(angle)
    
    local_screw_angles = []
    #Calculate rotation angles for (+1) to (n-1)
    fit_vector, fit_tilt = vector_of_best_fit(origins)
    for item in location_rotation_vectors:
        local_screw_tmp = np.rad2deg(rotation_angle(fit_vector, ref_axis, item))
        #print local_screw_tmp
        local_screw_angles.append(local_screw_tmp)

    return twist, bending_angles, height, rnou, origins, local_helix_axes, local_screw_angles


def rotation_angle(helix_vector, axis_vector, rotation_vector):
    reference_vector = np.cross(np.cross(helix_vector, axis_vector), helix_vector)
    second_reference_vector = np.cross(axis_vector, helix_vector)
    screw_angle = mdamath.angle(reference_vector, rotation_vector)
    alt_screw_angle = mdamath.angle(second_reference_vector, rotation_vector)
    updown = np.cross(reference_vector, rotation_vector)

    if not (np.pi < screw_angle < 3 * np.pi / 4):
        if screw_angle < np.pi / 4 and alt_screw_angle < np.pi / 2:
            screw_angle = np.pi / 2 - alt_screw_angle
        elif screw_angle < np.pi / 4 and alt_screw_angle > np.pi / 2:
            screw_angle = alt_screw_angle - np.pi / 2
        elif screw_angle > 3 * np.pi / 4 and alt_screw_angle < np.pi / 2:
            screw_angle = np.pi / 2 + alt_screw_angle
        elif screw_angle > 3 * np.pi / 4 and alt_screw_angle > np.pi / 2:
            screw_angle = 3 * np.pi / 2 - alt_screw_angle
        else:
            logger.debug("Big Screw Up: screw_angle=%g degrees", np.rad2deg(screw_angle))

    if mdamath.norm(updown) == 0:
        logger.warning("PROBLEM (vector is at 0 or 180)")

    helix_dot_rehelix = mdamath.angle(updown, helix_vector)

    #if ( helix_dot_rehelix < np.pi/2 and helix_dot_rehelix >= 0 )or helix_dot_rehelix <-np.pi/2:
    if (-np.pi / 2 < helix_dot_rehelix < np.pi / 2) or (helix_dot_rehelix > 3 * np.pi / 2):
        screw_angle = -screw_angle

    return screw_angle


def vector_of_best_fit(origins):
    origins = np.asarray(origins)
    centroids = center(origins)
    M = origins - centroids
    A = np.dot(M.transpose(), M)
    u, s, vh = np.linalg.linalg.svd(A)
    vector = vh[0]
    #Correct vector to face towards first residues
    rough_helix = origins[0] - centroids
    agreement = mdamath.angle(rough_helix, vector)
    if not (-np.pi / 2 < agreement < np.pi / 2):
        vector *= -1
    best_fit_tilt = mdamath.angle(vector, [0, 0, 1])
    return vector, best_fit_tilt

In [4]:
import MDAnalysis.analysis.helanal

c:\users\flurin hidber\anaconda3\envs\law\lib\site-packages\MDAnalysis\analysis\helanal.py:145: DeprecationWarning: This module is deprecated as of MDAnalysis version 1.0. It will be removed in MDAnalysis version 2.0.Please use (available in 2.0.0) MDAnalysis.analysis.helix_analysis instead.
  category=DeprecationWarning)


In [5]:
import MDAnalysis.analysis.helanal as hel

In [6]:
path = 'data/pdb/'

In [7]:
helenal = helanal_main(path+"2R4S.pdb", 
                           select="name CA and segid A and resid 310:320")

Analysing 10 CA atoms / 647 residues
CA positions: (10, 3)


In [8]:
helenal

{'Height': array([1.7989655 , 0.20282486, 0.17277493], dtype=float32),
 'Twist': array([107.67852  ,   4.9498477,   4.2698617], dtype=float32),
 'Residues/turn': array([3.349228  , 0.1508729 , 0.13077882]),
 'Local bending angles': array([ 6.629216, 16.324629, 14.890648, 12.518525], dtype=float32),
 'Unit twist angles': array([103.03859 , 104.849075, 103.274315, 104.60758 , 113.05057 ,
        109.82197 , 115.10754 ], dtype=float32),
 'Best fit tilt': 2.01212834975099,
 'Rotation Angles': array([  -3.39456423,   98.65267842, -156.15034133,  -53.05225337,
          51.62747702,  164.89826533,  -86.42084428,   28.48290816])}

In [75]:
helenal['Height']  # mean, stdev, abs dev

array([1.7989655 , 0.20282486, 0.17277493], dtype=float32)

In [76]:
helenal['Twist']  # mean, stdev, abs dev

array([107.67852  ,   4.9498477,   4.2698617], dtype=float32)

In [77]:
helenal['Residues/turn']  # mean, stdev, abs dev

array([3.349228  , 0.1508729 , 0.13077882])

In [78]:
helenal['Local bending angles'].shape  # array for computed angles (per residue)

(4,)

In [79]:
helenal['Unit twist angles'].shape

(7,)

In [80]:
helenal['Best fit tilt']

2.01212834975099

In [81]:
helenal['Rotation Angles'].shape

(8,)

In [100]:
class StructureAnalysis:
    def __init__(self,
                 processor):
        self.dfl = processor.dfl
        self.dfl_list = processor.dfl_list
        self.table = processor.table
        self.mappings = processor.mappings
        del processor
        
        # split the dfl list into a pdb-based datalist and an atom-wise dataframe datalist
        self.pdb_dfl = []
        self.res_dfl = []
        self.atom_dfl = []
        
    # ==========================================================================================================
    
    def __len__(self):
        return len(self.dfl)
    
    # ==========================================================================================================
    
    def _add_ref_axis_r(self, ref=[1.50, 2.50, 3.50, 4.50, 5.50, 6.50, 7.50]):
        # select group to average by (level)
        pass
    
    def _add_ref_axis_hook(self):
        # select group to average by (level)
        pass
    
    def _add_atomic_features(self):
        pass
    
    def _add_residual_features(self, mode='helenal'):
        # helenal featues
        if mode == 'helenal':
            self._add_helenal_1()
            self._add_helenal_2()
            self._add_helenal_3()
        elif mode == 'spatial':
            self._calc_res_spatial_1()  # area at axis_r-height of residue of the pocket
            self._calc_res_spatial_2()  
            self._calc_res_spatial_3()
        pass
    
    # ==========================================================================================================
    
    # ATOM-WISE FUNCTIONS  --> maybe necessary to detect what interactions take place (H-Bonds etc)
    
    def _calc_atom_spatial_1(self):
        pass

    def _calc_atom_spatial_2(self):
        pass

    def _calc_atom_spatial_3(self):
        pass

    # ==========================================================================================================
    
    # RESIDUE-WISE FUNCTIONS
    
    def _add_1(self):
        pass
    
    def _add_2(self):
        pass
    
    def _add_3(self):
        pass
    
    def get_insertion_score(self):
        pass
    
    def _calc_res_spatial_1(self):
        # how exposed / hollow is the residue to the binding pocket
        pass
    
    def _calc_res_spatial_2(self):
        # how much % of the binding pocket lies above/below        
        pass
    
    def _calc_res_spatial_3(self, res_vec):
        # the penetration of the residue vector
        pass
    
    def _calc_res_spatial_4(self):
        # the position within the shape of the cross-section of the 
        pass
    
    def _get_res_vec(self):
        # where does the residue AA sidechain point? How far?
        pass
    
    def _get_hydrophobicity(self):
        # where do I get this dict
        pass
    
    # ==========================================================================================================
    
    def distance_map(self,
                     indices=[],
                     x=[],
                     y=[],):
        pass   
    
    # ==========================================================================================================
        
    def plot(self,
             res_ids=[],
             aug=['all']):
        # plot the axis etc, overlay the residues, add binding pocket surface
        pass

# Adding other features (residue wise)

In [9]:
tsv_path = 'data/gauss.tsv'

In [10]:
def read_tsv(path):
    tsv_file = open(path)
    read_tsv = csv.reader(tsv_file, delimiter="\t")

    for row in read_tsv:
        print(row)

In [11]:
import csv

read_tsv(tsv_path)

['name', 'lambda', 'mu1', 'mu2', 'sigma1', 'sigma2', 'sigma3', 'sigma4']
['alpha', '0.3647574894801666', '297.61576315933524', '-40.58567040455446', '21.60327526214181', '-8.526550657622467', '-8.526550657622467', '43.102062848602984']
['beta', '0.24401588205844646', '233.28573477229165', '137.35381309008005', '379.2693676120664', '-242.4631534829163', '-242.4631534829163', '472.90244261585514']
['delta', '0.20257930337817134', '274.2675708989392', '-14.768732963735703', '394.2772687945463', '-189.41815948538473', '-189.41815948538473', '464.2947059927305']
['delta~"\'"', '0.051545648935406585', '75.58293609636182', '15.905626191703469', '309.5576733062913', '-307.8326184481008', '-307.8326184481008', '552.1217107954874']
['P[II]', '0.11137751721755791', '278.84201860815205', '138.15497776074636', '330.9390752766129', '20.116695134288957', '20.116695134288957', '212.34719109396855']
['P[II]~"\'"', '0.025724158930251078', '153.65336784343813', '190.43636665745018', '5737.390402395822', 